# Export ORIGINALNIH podataka za GeNIe

**Cilj:** Kreirati CSV s originalnim (nediskretiziranim) podacima za GeNIe.

GeNIe će sam napraviti binning i diskretizaciju.

**Što radimo:**
1. Učitamo originalni dataset
2. Dodamo ciljnu varijablu `target_G1` (1 = pad, 0 = prolaz)
3. Uklonimo G1, G2, G3 (sprječavanje data leakage)
4. Exportamo bez ikakve diskretizacije

In [1]:
import pandas as pd

# Učitaj originalni dataset
df = pd.read_csv('../dataset/student-por.csv', sep=';')

print(f"Učitano {len(df)} učenika s {len(df.columns)} varijabli")
print(f"\nOriginalne varijable: {list(df.columns)}")

Učitano 649 učenika s 33 varijabli

Originalne varijable: ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']


In [2]:
# Dodaj ciljnu varijablu
# target_G1 = 1 ako je G1 <= 10 (pad/rizik)
# target_G1 = 0 ako je G1 > 10 (prolaz)
df['target_G1'] = (df['G1'] <= 10).astype(int)

print("DISTRIBUCIJA CILJNE VARIJABLE:")
print(f"  Prolaz (target_G1 = 0): {(df['target_G1'] == 0).sum()} učenika")
print(f"  Pad    (target_G1 = 1): {(df['target_G1'] == 1).sum()} učenika")

DISTRIBUCIJA CILJNE VARIJABLE:
  Prolaz (target_G1 = 0): 397 učenika
  Pad    (target_G1 = 1): 252 učenika


In [3]:
# Ukloni G1, G2, G3 (sprječavanje data leakage)
df_export = df.drop(columns=['G1', 'G2', 'G3'])

print(f"KONAČNI DATASET:")
print(f"  Broj učenika: {len(df_export)}")
print(f"  Broj varijabli: {len(df_export.columns)}")
print(f"\nVarijable: {list(df_export.columns)}")

# Prikaz prvih 5 redaka
print("\nPRVIH 5 REDAKA:")
df_export.head()

KONAČNI DATASET:
  Broj učenika: 649
  Broj varijabli: 31

Varijable: ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'target_G1']

PRVIH 5 REDAKA:


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,target_G1
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,4,1
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,2,1
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,6,0
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,0,0
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,0,0


In [4]:
# Export u CSV (BEZ diskretizacije - GeNIe će to napraviti)
output_path = 'student_raw.csv'
df_export.to_csv(output_path, index=False)

print(f"Dataset exportan u '{output_path}'")
print(f"\nSLJEDEĆI KORACI U GENIE:")
print("  1. File → Open Data File → odaberi 'student_raw.csv'")
print("  2. Learning → Discretize → odaberi metodu za svaku varijablu")
print("  3. Learning Wizard → dodaj forsirane veze (failures, higher, school → target_G1)")
print("  4. Odaberi algoritam i nauči mrežu")

Dataset exportan u 'student_raw.csv'

SLJEDEĆI KORACI U GENIE:
  1. File → Open Data File → odaberi 'student_raw.csv'
  2. Learning → Discretize → odaberi metodu za svaku varijablu
  3. Learning Wizard → dodaj forsirane veze (failures, higher, school → target_G1)
  4. Odaberi algoritam i nauči mrežu


In [5]:
# Pregled tipova podataka (GeNIe će diskretizirati numeričke)
print("TIPOVI PODATAKA:")
print("="*60)

numeric_cols = df_export.select_dtypes(include=['number']).columns.tolist()
categorical_cols = df_export.select_dtypes(include=['object']).columns.tolist()

print(f"\nNUMERIČKE varijable ({len(numeric_cols)}) - GeNIe će diskretizirati:")
for col in numeric_cols:
    print(f"  • {col}: {df_export[col].nunique()} jedinstvenih vrijednosti")

print(f"\nKATEGORIČKE varijable ({len(categorical_cols)}) - već diskretne:")
for col in categorical_cols:
    print(f"  • {col}: {df_export[col].nunique()} kategorija")

TIPOVI PODATAKA:

NUMERIČKE varijable (14) - GeNIe će diskretizirati:
  • age: 8 jedinstvenih vrijednosti
  • Medu: 5 jedinstvenih vrijednosti
  • Fedu: 5 jedinstvenih vrijednosti
  • traveltime: 4 jedinstvenih vrijednosti
  • studytime: 4 jedinstvenih vrijednosti
  • failures: 4 jedinstvenih vrijednosti
  • famrel: 5 jedinstvenih vrijednosti
  • freetime: 5 jedinstvenih vrijednosti
  • goout: 5 jedinstvenih vrijednosti
  • Dalc: 5 jedinstvenih vrijednosti
  • Walc: 5 jedinstvenih vrijednosti
  • health: 5 jedinstvenih vrijednosti
  • absences: 24 jedinstvenih vrijednosti
  • target_G1: 2 jedinstvenih vrijednosti

KATEGORIČKE varijable (17) - već diskretne:
  • school: 2 kategorija
  • sex: 2 kategorija
  • address: 2 kategorija
  • famsize: 2 kategorija
  • Pstatus: 2 kategorija
  • Mjob: 5 kategorija
  • Fjob: 5 kategorija
  • reason: 4 kategorija
  • guardian: 3 kategorija
  • schoolsup: 2 kategorija
  • famsup: 2 kategorija
  • paid: 2 kategorija
  • activities: 2 kategorija
  • nu